In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import dataframe as df
import numpy as np
import time

#options = Options()
#options.binary_location = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
chromedriver='C:/Users/default.DESKTOP-FVT3076/Desktop/crawl/chromedriver.exe'
#chromedriver = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
driver = webdriver.Chrome(chromedriver)
time.sleep(2)

In [ ]:
import pandas as pd

# 
df = pd.read_csv('성북구_음식점.csv', sep=',', encoding='CP949')

df = df[['업소명', '소재지도로명', '업태명', '주된음식', '행정동명', '소재지전화번호']]
df.columns = ['name', 'address', 'cate1', 'cate2', 'dong', 'phone']
df = df.drop_duplicates(['name'], keep='first')
# 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.
# df['cate_mix'] = df['cate1'] + df['cate2']

print(df)

df['naver_keyword'] = df['dong'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''


# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(0.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

In [32]:
# 크롤링 파일 저장

import os
from datetime import datetime

today = str(datetime.date(datetime.today()))
file = today + '_test.csv'

if os.path.isfile(file):
    os.remove(file)
df.to_csv(file, encoding='utf-8')
# df.to_csv(today + '_test.csv', encoding='cp949')
print(file)

In [26]:
from tqdm.notebook import tqdm

naver_map_type_list = []
blog_review_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 블로그 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)

for i, url in enumerate(tqdm(df['naver_map_url'])):

    driver.get(url)
    sub_driver.get(url+"/review/ugc")
    time.sleep(0.5)


    try:

        # 간단 정보 가져오기
        
        # 네이버 지도의 유형 분류
        naver_map_type = driver.find_element_by_css_selector("#_title > span._3ocDE").text

        # 블로그 리뷰 수
        blog_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(3) > a > em").text

        # 블로그 별점 점수
        star_review_stars = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em").text

        # 블로그 별점 평가 수
        star_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(2) > a > em").text
       

        # 블로그 리뷰 텍스트 가져오기
        review_text_list = [] # 임시 선언

        
        # 네이버 지도 블로그 리뷰 탭은 동적 웹사이트의 순서가 주문하기, 메뉴보기 등의 존재 여부로 다르기 때문에 css selector가 아니라 element 찾기로 진행
        review_text_crawl_list = sub_driver.find_elements_by_class_name("_2CbII")
        
        # find element's' 메소드를 통해 가져온 내용은 리스트로 저장되고, 리스트 타입을 풀어서(for문 사용) 임시 데이터에 모아 두어야 한다
        for review_crawl_data in review_text_crawl_list:
            review_text_list.append(review_crawl_data.find_element_by_tag_name('div').text)
        
        # 그 리스트에 저장된 텍스트 (한 식당에 대한 여러 리뷰들)를 한 텍스트 덩어리로 모아(join)줍니다.
        review_text = ','.join(review_text_list)


        blog_review_list.append(review_text)

        naver_map_type_list.append(naver_map_type)
        blog_review_qty_list.append(blog_review_qty)
        naver_map_star_review_stars_list.append(star_review_stars)
        naver_map_star_review_qty_list.append(star_review_qty)

    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        blog_review_list.append('null')  
        naver_map_type_list.append('null')
        blog_review_qty_list.append('null')
        naver_map_star_review_stars_list.append('null')
        naver_map_star_review_qty_list.append('null')


        
driver.quit()
sub_driver.quit()


df['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
df['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
df['naver_star_point_qty'] = naver_map_star_review_qty_list  # 네이버 상세페이지에서 별점 평가를 한 횟수
df['naver_blog_review_txt'] = blog_review_list  # 네이버 상세페이지에 나온 블로그 리뷰 텍스트들
df['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수

14행 문제가 발생
31행 문제가 발생
32행 문제가 발생
37행 문제가 발생
38행 문제가 발생
47행 문제가 발생
65행 문제가 발생
73행 문제가 발생
79행 문제가 발생



In [27]:
df

,name,address,cate1,cate2,dong,phone,cate_mix,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_star_point_qty,naver_blog_review_txt,naver_blog_review_qty
1,김태완의초밥좋은날,서울특별시 성북구 동소문로20길 28-4 (동소문동5가 2층),식육(숯불구이),삼겹살,삼선동,02 923 8628,식육(숯불구이)삼겹살,삼선동%20김태완의초밥좋은날,https://m.place.naver.com/restaurant/21816603,"초밥,롤",4.33,932,돈암동스시 퀄리티가 좋은 김태완의 초밥좋은날 큰언니 작은언니와 함께 돈암시장 근처 ...,483
4,만복식당(성신여대점),서울특별시 성북구 동소문로20길 28-4 (동소문동5가 2층),식육(숯불구이),삼겹살,삼선동,02 923 8628,식육(숯불구이)삼겹살,삼선동%20만복식당(성신여대점),https://m.place.naver.com/restaurant/1932252419,돼지고기구이,4.42,141,만복식당은 2층에 있어요. 내부는 크고 사이드에는 여름엔 창문을 열고 고기를 먹을 ...,42
8,유자유 김치떡볶이,서울특별시 성북구 인촌로24길 48 2층 (안암동5가),한식,김치떡볶이,안암동,02 928 7754,한식김치떡볶이,안암동%20유자유 김치떡볶이,https://m.place.naver.com/restaurant/20470283,떡볶이,4.48,338,땅거미가 내려앉기 시작하는 저녁 7시. 유명한 안암역 맛집을 찾아 고려대에 왔습니다...,645
11,정릉곤드레밥,서울특별시 성북구 보국문로18가길 5-13 (정릉동),한식,곤드레밥,정릉제4동,NaN,한식곤드레밥,정릉제4동%20정릉곤드레밥,https://m.place.naver.com/restaurant/1338424116,한식,4.44,72,한국인 중에 비빔밥을 싫어하는 사람을 별로 없을듯 싶다. 오늘은 정릉에 계시는 지인...,11
12,만복식당,서울특별시 성북구 개운사길 34 지1층 (안암동5가),한식,삼겹살,안암동,NaN,한식삼겹살,안암동%20만복식당,https://m.place.naver.com/restaurant/1541937450,한식,4.5,130,"안암동, 고려대 근처에서 점심 먹기 좋은 만복식당 ! 고려대 화정체육관으로 콘서트 ...",22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,메밀촌,서울특별시 성북구 화랑로25길 17 (상월곡동),한식,만두국,월곡제2동,02 916 2141,한식만두국,월곡제2동%20메밀촌,https://m.place.naver.com/restaurant/11590804,한식,4.45,271,"최근 우리의 일상을 바꾼 코로나19. 손씻기, 마스크착용, 외출자제, 거리두기 캠페...",31
195,금왕돈까스,서울특별시 성북구 성북로 138 (성북동),경양식,돈까스,성북동,0207642691,경양식돈까스,성북동%20금왕돈까스,https://m.place.naver.com/restaurant/11591181,돈가스,4.19,"1,528",지난 주말 과거 수요미식회를 비롯한 다양한 음식 프로그램에서 맛집으로 소개된 성북구...,796
196,큰솥감자탕,서울특별시 성북구 동소문로24길 31 (동선동1가1층),한식,돼지갈비,동선동,02 9212302,한식돼지갈비,동선동%20큰솥감자탕,https://m.place.naver.com/restaurant/73321843,감자탕,4.48,93,큰솥감자탕삼겹살 02. 921. 2302 서울 성북구 동소문로24길 31 매일 09...,22
198,한강,서울특별시 성북구 창경궁로43길 17 (성북동1가),한식,국수,성북동,02 7474004,한식국수,성북동%20한강,https://m.place.naver.com/restaurant/12011134,국수,4.14,43,성북동 맛집 성북동 소박한 칼국수와 바삭한 파전 맛집 한강 4호선 한성대역 근처 창...,27


In [50]:
# %%cmd
# head C:/Users/default.DESKTOP-FVT3076/Desktop/data/data.json
# 안되니까 git bash로 확인
# 한줄 데이터라 byte 단위로 json 구조 확인
# head $ head data.json -c 1000

Microsoft Windows [Version 10.0.17134.2145]
(c) 2018 Microsoft Corporation. All rights reserved.

C:\Users\default.DESKTOP-FVT3076\Desktop\crawl>head C:/Users/default.DESKTOP-FVT3076/Desktop/data/data.json

C:\Users\default.DESKTOP-FVT3076\Desktop\crawl>

'head'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [54]:
#데이터가 너무 커서 블록단위로 메모리에 올리는 ijson 사용
import ijson
import pandas

with open('C:/Users/default.DESKTOP-FVT3076/Desktop/data/data.json','r',encoding='utf-8') as f:
    obj = ijson.items(f, 'columns.item')
    columns = list(obj)
    print(columns[0])

IndexError: list index out of range